In [1]:
!pip install openai

from openai import OpenAI  # Replace Groq with OpenAI
import networkx as nx

import numpy as np
from scipy import stats as sps
from matplotlib import pyplot as plt
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import time

class Agent:
    def __init__(self, api_key, model, description=None, api_type="Groq", max_retries=5, retry_delay=50):
        self.client = OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")
        self.model = model
        self.description = description
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.history = [
            {"role": "system", "content": f"{self.description}"}
        ]

    def _generate_response(self, messages):
        print(f"# call with {messages}")
        response = None
        try:
          response = self.client.chat.completions.create(
              messages=messages,
              model=self.model,
              temperature=0,
              top_p=0.5
          )
        except Exception as e:
          print(f"# Error: {e}")
        print(f"# raw: {response}")

        # Check for API error
        if getattr(response, "error", None):
            print(f"# API error {response.error.get('code')}: {response.error.get('message')}")
            return None

        # Check if choices exist
        if not getattr(response, "choices", None):
            print("# No choices returned")
            return None

        content = getattr(response.choices[0].message, "content", None)
        if not content:
            print("# Empty content returned")
            return None

        return content

    def stateful_chat(self, user_message, history_message = ""):
        self.history.append({"role": "user", "content": user_message})
        response = None
        for attempt in range(self.max_retries):
            response = self._generate_response(self.history)
            if response is not None:
                try:
                    tmp = [float(x) for x in response.split()]
                    if len(tmp) != 3:
                        raise ValueError("Response does not contain exactly 3 numbers")
                except ValueError:
                    # Parsing failed or wrong number of floats -> retry
                    print(f"# Retry {attempt+1}/{self.max_retries} after {self.retry_delay}s...")
                    time.sleep(self.retry_delay)
                    continue  # go to next retry attempt

                # If we got here, parsing succeeded
                print(f"#{response}")
                self.history.pop()
                if history_message != "":
                    self.history.append({"role": "user", "content": history_message})
                return response
            print(f"# Retry {attempt+1}/{self.max_retries} after {self.retry_delay}s...")
            time.sleep(self.retry_delay)

        self.history.pop()
        return response

    def reset_history(self):
        self.history = [self.history[0]]

In [4]:
N=12

In [5]:
network = np.array([[0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1],
 [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
 [1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
 [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1],
 [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
 [0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0]])
weights = np.array([[0.333, 0.333, 0.334] for i in range(N)])

In [6]:
edges = [[] for i in range(N)]
for i in range(N):
    for j in range(i + 1, N):
        if (network[i][j] == 1):
            edges[i].append(j)
            edges[j].append(i)
    edges[i].append(i)
edges

[[3, 5, 6, 9, 0],
 [7, 8, 10, 11, 1],
 [4, 7, 9, 10, 11, 2],
 [0, 4, 6, 8, 11, 3],
 [2, 3, 5, 8, 11, 4],
 [0, 4, 6, 7, 9, 10, 11, 5],
 [0, 3, 5, 8, 6],
 [1, 2, 5, 7],
 [1, 3, 4, 6, 11, 8],
 [0, 2, 5, 10, 11, 9],
 [1, 2, 5, 9, 10],
 [1, 2, 3, 4, 5, 8, 9, 11]]

In [26]:
import pickle
with open("data/originaltags.pkl", "rb") as f:
    original_tags = pickle.load(f)

In [24]:
import pickle
with open("data/Moviegroups.pkl", "rb") as f:
    movie_group = pickle.load(f)

### Run LLMs

In [11]:
OPENAI_API_KEY = "..."

In [12]:
def normalize(p, eps = 0.011):
    p = np.array(p, dtype=float) + eps
    p = p / p.sum()
    return p

In [13]:
description1 ="You are participating in a movie guessing game with three possible movies.  Your previous belief is given; as well as a clue related to the correct movie. Your task is to output your new belief probabilites. Return ONLY three space-separated decimal numbers. NO explanations, NO text, NO formatting.",

In [14]:
description2 = "You are participating in a movie guessing game with three possible movies. Your previous belief and your neighbors' beliefs are given. Your neighbors have seen different clues than you. Your task is to output your new belief probabilites. Return ONLY three space-separated decimal numbers. NO explanations, NO text, NO formatting.",

In [15]:
agents1 = []
agents2 = []
for desc_id in range(N):
    agent1 = Agent(
        api_key=OPENAI_API_KEY,
        model="openai/gpt-oss-120b",
        api_type="OpenRouter",
        description=description1
    )
    agent2 = Agent(
        api_key=OPENAI_API_KEY,
        model="openai/gpt-oss-120b",
        api_type="OpenRouter",
        description=description2
    )
    agents1.append(agent1)
    agents2.append(agent2)

In [17]:
shuffled_time = np.random.RandomState(seed=42).permutation(20)
shuffled_time

array([ 0, 17, 15,  1,  8,  5, 11,  3, 18, 16, 13,  2,  9, 19,  4, 12,  7,
       10, 14,  6])

In [18]:
num_samples = 10
T = 15

In [28]:
# guess = np.zeros((1000, 100, N, top_n)) # mu from the algorithm
# intermediary_guess = np.zeros((1000, 100, N, top_n)) # psi from the algorithm

In [20]:
with open("data/interguess", "rb") as f:
    intermediary_guess = np.load(f)
with open("data/guess", "rb") as f:
    guess = np.load(f)

In [22]:
real_class = 0

In [31]:
def get_clue(real_movie, group_number, time):
    clue_id = movie_group[real_movie][group_number][time][0]
    clue = original_tags[clue_id]
    return clue

for sample in range(num_samples):
    for k in range(N):
        agents1[k].reset_history()
        agents2[k].reset_history()
    prev_guess = [[0.333, 0.333, 0.334] for i in range(N)]

    for id_t in range(T):
        t = shuffled_time[id_t]

        for k in range(N):
            clue = get_clue(real_class, k, t)
            if (intermediary_guess[sample][id_t][k][0] == 0):
                head_prompt = f"""New clue: \"{clue}\". Your own belief from the previous round over [Pulp Fiction, Forrest Gump, Jurassic Park] was: {prev_guess[k]}.
Based on the new clue and your previous belief, what is your belief over the classes [Pulp Fiction, Forrest Gump, Jurassic Park]? Return ONLY EXACTLY three space-separated decimal numbers. NO explanations, NO text, NO formatting"""
                response = agents1[k].stateful_chat(head_prompt)
                intermediary_guess[sample][id_t][k] = normalize(np.array([float(x) for x in response.split()]))
            print(f"Sample {sample}, time {id_t}, agent {k}.    {clue}: {intermediary_guess[sample][id_t][k]}")
        for k in range(N):
            if (guess[sample][id_t][k][0] == 0):
                head_prompt = f"Your own belief from the previous round over [Pulp Fiction, Forrest Gump, Jurassic Park] was: {intermediary_guess[sample][id_t][k]}.\n Your neighbors' beliefs for [Pulp Fiction, Forrest Gump, Jurassic Park] are\n"
                for neighbor in edges[k]:
                    if (neighbor != k):
                        head_prompt += f"Neighbor {neighbor}: {intermediary_guess[sample][id_t][neighbor]}\n"
                head_prompt += f"Based on the provided information, what is your current probability belief over [Pulp Fiction, Forrest Gump, Jurassic Park]? Return ONLY EXACTLY three space-separated decimal numbers. NO explanations, NO text, NO formatting."

                response = agents2[k].stateful_chat(head_prompt)
                guess[sample][id_t][k] = normalize(np.array([float(x) for x in response.split()]))
            print(f"\t agent {k}. Update: {guess[sample][id_t][k]}")
        prev_guess = guess[sample][id_t]

Sample 0, time 0, agent 0.    mafia: [0.78509197 0.10745402 0.10745402]
Sample 0, time 0, agent 1.    weird: [0.63988383 0.18005808 0.18005808]
Sample 0, time 0, agent 2.    unlikely friendships: [0.20425944 0.59148112 0.20425944]
Sample 0, time 0, agent 3.    surprise ending: [0.59148112 0.20425944 0.20425944]
Sample 0, time 0, agent 4.    whimsical: [0.10745402 0.68828654 0.20425944]
Sample 0, time 0, agent 5.    drinking: [0.59108527 0.20445736 0.20445736]
Sample 0, time 0, agent 6.    forceful: [0.44627299 0.20425944 0.34946757]
Sample 0, time 0, agent 7.    cynical: [0.68828654 0.15585673 0.15585673]
Sample 0, time 0, agent 8.    morality: [0.20425944 0.59148112 0.20425944]
Sample 0, time 0, agent 9.    twists & turns: [0.59148112 0.20425944 0.20425944]
Sample 0, time 0, agent 10.    not funny: [0.20425944 0.20425944 0.59148112]
Sample 0, time 0, agent 11.    bleak: [0.78509197 0.10745402 0.10745402]
	 agent 0. Update: [0.59252903 0.18967855 0.21779242]
	 agent 1. Update: [0.49889

	 agent 4. Update: [0.50685479 0.31283833 0.18030687]
	 agent 5. Update: [0.55679017 0.3020359  0.14117393]
	 agent 6. Update: [0.61221215 0.29011764 0.09767021]
	 agent 7. Update: [0.35078059 0.49183203 0.15738737]
	 agent 8. Update: [0.49802816 0.33458107 0.16739076]
	 agent 9. Update: [0.48978544 0.36266561 0.14754894]
	 agent 10. Update: [0.42318025 0.49471411 0.08210563]
	 agent 11. Update: [0.489402   0.36135106 0.14924694]
Sample 7, time 6, agent 0.    gangster: [0.91540174 0.05830591 0.02629235]
Sample 7, time 6, agent 1.    dystopic future: [0.20309478 0.29497099 0.50193424]
Sample 7, time 6, agent 2.    gypsy accent: [0.68115802 0.26229667 0.05654531]
Sample 7, time 6, agent 3.    dark hero: [0.96902227 0.01645692 0.01452081]
Sample 7, time 6, agent 4.    cult film: [0.88425944 0.07056147 0.04517909]
Sample 7, time 6, agent 5.    snakes: [0.54965167 0.30303572 0.14731261]
Sample 7, time 6, agent 6.    adapted from:book: [0.0106486  0.73488383 0.25446757]
Sample 7, time 6, age

In [30]:
with open("data/interguess", "rb") as f:
    intermediary_guess = np.load(f)
with open("data/guess", "rb") as f:
    guess = np.load(f)